# Title
Regression Project (Store Sales -- Time Series Forecasting)

# Project Description
This is a time series forecasting problem. In this project, we will predict store sales on data from Corporation Favorita, a large Ecuadorian-based grocery retailer.

Specifically, we are to build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores.

The training data includes dates, store, and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models

# Hypothesis
## Null Hypothesis, HO
Series is non-stationary
## AlternativeHypothesis, H1
Series is stationary
# Questions
1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. What analysis can we get from the date and its extractable features?

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

ADDITIONAL QUESTIONS

8. What is the trend of sales over time?
9. What is the trend of transactions over time?
10. Highest and lowest performing stores in terms of sales
11. Highest performing family of products


# Importation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import matplotlib.dates as mdates
from pandas_profiling import ProfileReport
import plotly.graph_objects as go
%matplotlib inline

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

import warnings
warnings.filterwarnings("ignore")

# Data Loading

In [ ]:
holidays=pd.read_csv("store-sales-time-series-forecasting/holidays_events.csv")
oil=pd.read_csv("store-sales-time-series-forecasting/oil.csv")
sample=pd.read_csv("store-sales-time-series-forecasting/sample_submission.csv")
stores=pd.read_csv("store-sales-time-series-forecasting/stores.csv")
test=pd.read_csv("store-sales-time-series-forecasting/test.csv")
train=pd.read_csv("store-sales-time-series-forecasting/train.csv", parse_dates =['date'])
transactions=pd.read_csv("store-sales-time-series-forecasting/transactions.csv")

# Exploratory Data Analysis: EDA
# Dataset overview

In [ ]:
# profile_data = ProfileReport(train, title ="Profiling Report")
# profile_data

Our date starts from January 2013 till October 2017

Sales has a strong positive correlation with onpromotion, so we'll focus on sales, id and onpromotion since they correlate with one another the most

No missing values in train data

In [ ]:
test.info()

In [ ]:
transactions.info()

we can see that the date column is in object instead of datetime

so we convert the datatypes so we can explore them further

In [ ]:
#converting date columns to datetime
def to_dateTime(df):
    df['date'] = pd.to_datetime(df['date'])

to_dateTime(transactions)
to_dateTime(test)
to_dateTime(oil)
to_dateTime(holidays)

In [ ]:
#indexing our date column
train=train.set_index(['date'])
test=test.set_index(['date'])
transactions=transactions.set_index(['date'])

In [ ]:
train.head()

In [ ]:
train['family'].unique()

In [ ]:
# train['family'] = train['family'].apply(
#     lambda x: str(x).replace('GROCERY I','GROCERY II') if '₹' in x
# else x) 

In [ ]:
transactions.head()

In [ ]:
holidays.tail(),holidays.shape

In [ ]:
#drop unnecessary columns in holidays
holidays.drop(
    columns=['locale', 'locale_name'],
    inplace=True
    )
# delete rows with transferred as true
transferred_true = holidays[ (holidays['transferred'] == True)].index
holidays.drop(transferred_true , inplace=True)

In [ ]:
#the id column doesn't give additional info, so we drop it
train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)

In [ ]:
oil.head()

In [ ]:
stores.head()

let's check for missing values

In [ ]:
print(transactions.isnull().sum())
print(test.isnull().sum())
print(train.isnull().sum())
print(stores.isnull().sum())

In [ ]:
print(sample.isnull().sum())
print(oil.isnull().sum())
print(holidays.isnull().sum())

our oil data has 43 missing values, we'll deal with them

In [ ]:
null_data = oil[oil.isnull().any(axis=1)]
null_data

In [ ]:
# oil=oil.dropna()
# oil.isnull().sum()

### merging our data
we want to merge the transactions with our train data, so we can use it to train our model

In [ ]:
train.shape

In [ ]:
#merging transactions with our train data
merged=pd.merge(
    train.reset_index(), transactions.reset_index(),
    how='outer', 
    on=['date', 'store_nbr']
    ).set_index('date')
merged.head()

In [ ]:
#merging merged with holidays
merged2=pd.merge(
    merged.reset_index(), holidays,
    how='outer', 
    on=['date']
    ).set_index('date')
merged2.head()

In [ ]:
#merging merged2 with oil data
merged3=pd.merge(
    merged2.reset_index(), oil,
    how='outer', 
    on=['date']
    ).set_index('date')

#renaming our oil column 
merged3.rename(
    columns = {'dcoilwtico':'oil_price'}, 
    inplace = True
    )
merged3.head()

In [ ]:
# # Specify the data columns we want to include (i.e. exclude Year, Month, Weekday Name)
# data_columns = ['store_nbr', 'sales', 'transactions', 'family', 'onpromotion']
# # Resample to daily frequency, aggregating with sum
# merged_daily_sum = merged[data_columns].resample('D').sum()

## Univariate Analysis

8. What is the trend of sales over time?

In [ ]:
## Visualizing sales in train data
plt.figure(figsize=(12,5))
plt.title('Sales over Time')
train['sales'].plot(linewidth = 0.5)

sales in our train data is seasonal. There is no trend

9. What is the trend of transactions over time?

In [ ]:
# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(transactions.index), y=list(transactions.transactions)))

# Set title
fig.update_layout(
    title_text="Transactions over Time"
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

we get a spike in transactions every December 23rd & around May 10th throughout, this shows seasonality, but no trend
### ADF Test

In [ ]:
adfuller(transactions.transactions)

Falied to reject Null Hypothesis: Since p-value is 1.85e-29 (<0.05), Series is non-stationary
## KPSS Test

In [ ]:
kpss(transactions.transactions)

p-value is 0.01(<0.05) meaning our series is non-stationary

In [ ]:
## Visualizing oil prices in merged data
plt.figure(figsize=(12,5))
plt.title('Oil prices over Time')
merged3['oil_price'].plot.area(linewidth = 0.5)

## Bivariate Analysis

10. Highest and lowest performing stores in terms of sales

In [ ]:
top_stores_sales = (
    merged.groupby("store_nbr")["sales"]
    .sum()
    .reset_index()
    .sort_values(by="sales",ascending=False)
)

top_stores_transactions = (
    merged.groupby("store_nbr")["transactions"]
    .sum()
    .reset_index()
    .sort_values(by="transactions",ascending=False)
)

topSS=top_stores_sales.iloc[:5]
topST=top_stores_transactions.iloc[:5]

In [ ]:
#Plotting stores Vs sales and Transactions
fig, axes = plt.subplots(1, 2, figsize=(10, 3), sharey=True)
fig.suptitle('Top Stores by Sales and Transactions')

# Bulbasaur
sns.barplot(ax=axes[0], x=topSS.store_nbr, y=topSS.sales)
# axes[0].set_title(bulbasaur.name)

# Charmander
sns.barplot(ax=axes[1], x=topST.store_nbr, y=topST.transactions)
# axes[1].set_title(charmander.name)

11. Highest performing family of products

In [ ]:
top_family_by_sales = (
    merged.groupby("family")["sales"]
    .sum()
    .reset_index()
    .sort_values(by="sales",ascending=False)
)

topFS=top_family_by_sales.iloc[:5]

In [ ]:
#Plotting stores Vs sales 
fig = plt.figure(figsize=(10,3))
plt.title("Top Family by sales")
sns.barplot(data=topFS, x='family', y='sales', palette='Blues_d')
fig.show()

sns.set(font_scale = 1)
plt.xticks(rotation=45)

## Issues with the data
1. missing values in our oil data before merging
2. We need some categorical columns for our model
3. The data has a lot of missing values after merging

## How to fix them
1. We leave them for now
2. We have to do some encoding
3. For time series data, we most likely fill with the value closest to it, but overall, we have to be very careful

# Feature Processing & Engineering
## Drop Duplicates

In [ ]:
dup = merged3.loc[merged3.duplicated(),:]
print(dup.shape)
dup.tail(60)

We've looked at the duplicate rows,

they contain mostly null values

they aren't useful, so we drop them

In [ ]:
# Use pandas.DataFrame.drop_duplicates method
merged3.drop_duplicates(keep='first', inplace=True)

## Impute Missing Values

first, lets have a quick overview before deciding how to handle missing values

In [ ]:
# sns.heatmap(merged3.isnull(), cbar=False)

In [ ]:
merged3.isnull().sum()

let's interpolate the 1st 4 columns

In [ ]:
merged3['store_nbr'] = merged3['store_nbr'].interpolate(method='linear')
imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
merged3['family'] = imputer.fit_transform(merged3[['family']])
merged3['sales'] = merged3['sales'].interpolate(method='time')
merged3['onpromotion'] = merged3['onpromotion'].interpolate(method='linear')

since non-holidays are regular days, let's impute these 3 columns

In [ ]:
#Fill missing holiday rows with normal days
def replacer(column, text):
    imputer = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=text)
    merged3[column] = imputer.fit_transform(merged3[[column]])

replacer('type','Regular')
replacer('description','Regular Day')
replacer('transferred',False)

the remaining columns have more null values, so let's check which method is best

by creating function that checks r2 scores of interpolation methods

In [ ]:
def interpolation_method_checker(df):
    # select a series of non-null values
    df_not_null = df[~df.isna()]

    # Randomly set a percentage of this data as missing
    p = 0.4 #percentage missing data required (40 percent)
    df = pd.DataFrame(np.random.randint(0,100,size=(10,10)))
    mask = np.random.choice([True, False], size=df_not_null.shape, p=[p,1-p])
    df_null = df_not_null.mask(mask)
    
    # Try different interpolation methods
    time = df_null.interpolate(method='time')
    linear = df_null.interpolate(method='linear')
    slinear = df_null.interpolate(method='slinear')
    pad = df_null.interpolate(method='pad')

    # check r2 scores
    from sklearn.metrics import r2_score
    t = r2_score(df_not_null, time)
    l = r2_score(df_not_null, linear)
    sl = r2_score(df_not_null, slinear)
    p = r2_score(df_not_null, pad)

    print(t,l,sl,p)

In [ ]:
interpolation_method_checker(merged3['transactions'])

In [ ]:
interpolation_method_checker(merged3['oil_price'])

In [ ]:
# using linear interpolation
merged3['transactions'] = merged3['transactions'].interpolate(method='linear')
merged3['transactions'] = merged3['transactions'].bfill()

merged3['oil_price'] = merged3['oil_price'].interpolate(method='linear')
merged3['oil_price'] = merged3['oil_price'].bfill()

In [ ]:
# sns.heatmap(merged3.isnull(), cbar=False)

In [ ]:
merged3.isna().sum()

## New Features Creation

In [ ]:
## Use pandas' powerful time-based indexing to analyze data
test['Year'] = test.index.year
test['Month'] = test.index.month
test['Weekday Name'] = test.index.day_name()

merged3['Year'] = merged3.index.year
merged3['Month'] = merged3.index.month
merged3['Weekday Name'] = merged3.index.day_name()

In [ ]:
train1=merged3
train1.shape

## Dataset Splitting

In [ ]:
# Use train_test_split with a random_state, and add stratify for Classification

## Features Encoding

In [ ]:
train1.head(2)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

train_labels=train1.drop('sales',axis=1)
cat_attribs=["family", 'type', 'description', 'transferred', 'Weekday Name']
train_num=train1.drop(["family", 'type', 'description', 'transferred', 'Weekday Name'], axis=1)
num_attribs=list(train_num)


numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder())
    ]
)
full_pipeline = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, cat_attribs),
        ("num", numeric_transformer, num_attribs)
    ]
)
train_prepared=full_pipeline.fit_transform(train1)

In [ ]:
train_prepared.shape

## Features Scaling


# Machine Learning Modeling 

## Simple Model #001

### Create the Model

In [ ]:
# Code here

### Train the Model

In [ ]:
# Use the .fit method